# Milestone 1: Tackling big data on your laptop

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Download the data
Here we download the data from figshare and extract the zip file programmatically.

In [2]:
%cd ~/MDS_B6/525/DSCI_525_Group_8/data/

C:\Users\maria\MDS_B6\525\DSCI_525_Group_8\data


In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])


CPU times: total: 7.69 s
Wall time: 4min 13s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 17.8 s
Wall time: 34.5 s


## Combining Data CSVs

Here we will combine the extracted CSVs into one CSV using pandas and compare runtimes between team members.

In [7]:
%cd ~/MDS_B6/525/DSCI_525_Group_8/data/figshare/

C:\Users\maria\MDS_B6\525\DSCI_525_Group_8\data\figshare


In [8]:
%%time
## here we are using a normal python way for merging the data 

files = glob.glob('*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("../combined_data.csv")

CPU times: total: 6min 59s
Wall time: 11min 51s


In [9]:
# check that model column values are in proper format
df.head(1)

lat_min  lat_max  lon_min  lon_max  rain (mm/day)  \
time                                                                     
1889-01-01 12:00:00   -36.25    -35.0  140.625    142.5   3.293256e-13   

                          model  
time                             
1889-01-01 12:00:00  ACCESS-CM2

Below, we compare the run times of combining CSVs on team members' laptops.

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Lisa   |        Windows 11          |   16 GB  |    11th Gen Intel(R) Core(TM) i5       |  Yes      |    10min 19s        |
| Dudu    |      MacOS Ventura Version 13.2.1            | 16 GB    |   2.3 GHz 8-Core Intel Core i9        |   Yes     |     5min 16s       |
| Marian    |       Windows 11           | 16 GB    |    12th Gen Intel(R) Core(TM) i7    |    Yes    |    11min 51s        |
| Mehwish   |         Windows 11         |  16GB   |    11th Gen Intel(R) Core(TM) i5       |   Yes     |      15min 42s      |

## Reduce Memory Usage & EDA

In [10]:
%cd ~/MDS_B6/525/DSCI_525_Group_8/

C:\Users\maria\MDS_B6\525\DSCI_525_Group_8


Below, we can see that we have over 62 million entries, and the data types of the numerical columns are all `float64`. 

In [11]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62513863 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB
CPU times: total: 0 ns
Wall time: 16.7 ms


> We can reduce memory usage by a) loading in data in smaller chunks, and b) converting `float64` to `float32`.  Below, we test out these memory reduction methods and time the EDA process, where we determine the value counts of each model.

In [12]:
%%time

# a) loading in by chunks + EDA
counts = pd.Series(dtype=int)

for chunk in pd.read_csv("data/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
    
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int32
CPU times: total: 1min 8s
Wall time: 2min 5s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Lisa   |        Windows 11          |   16 GB  |    11th Gen Intel(R) Core(TM) i5       |  Yes      |    57.2s      |
| Dudu    |      MacOS Ventura Version 13.2.1            | 16 GB    |   2.3 GHz 8-Core Intel Core i9        |   Yes     |     1min 2s     |
| Marian    |       Windows 11           | 16 GB    |    12th Gen Intel(R) Core(TM) i7    |    Yes    |   2min 5s       |
| Mehwish   |         Windows 11         |  16GB   |    11th Gen Intel(R) Core(TM) i5       |   Yes     |    2 min  45s      |

In [13]:
%%time

# b) convert numerical columns to float32
df = pd.read_csv("data/combined_data.csv").astype('float32', errors='ignore')

# EDA
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
CPU times: total: 1min 18s
Wall time: 2min 32s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Lisa   |        Windows 11          |   16 GB  |    11th Gen Intel(R) Core(TM) i5       |  Yes      | 1 min 15s         |
| Dudu    |      MacOS Ventura 13.2.1            | 16 GB    |   2.3 GHz 8-Core Intel Core i9        |   Yes     |    1min 15s      |
| Marian    |       Windows 11           | 16 GB    |    12th Gen Intel(R) Core(TM) i7    |    Yes    |   2min 32s       |
| Mehwish   |         Windows 11         |  16GB   |    11th Gen Intel(R) Core(TM) i5       |   Yes     |    2min 58s       |

## Transfer Dataframe to R & EDA

*Explain transfer method of choice here*

## Discussion of Challenges

*Discuss here.*